In [148]:
from tqdm import tqdm

import konlpy
import pickle
import re

from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Twitter
hannanum = Hannanum()
kkma = Kkma()
twitter = Twitter()

import xgboost
import lightgbm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

---

# Train (강남3구 - 치킨)

### 강남구

In [199]:
def make_train_dataset(gu_list, category):
    train_df = pd.DataFrame()
    
    for gu in gu_list:
        if gu=='송파구':
            dong_list=['가락동','거여동','마천동','문정동','방이동','삼전동','석촌동','송파동','오금동','오륜동','잠실동','장지동','풍납동']
        elif gu=='강남구':
            dong_list=['역삼동','개포동','청담동','삼성동','대치동','신사동','논현동','압구정동','세곡동','자곡동','율현동','일원동','수서동','도곡동']
        elif gu=='서초구':
            dong_list=['반포동','내곡동','방배동','서초동','신원동','양재동','염곡동','우면동','원지동','잠원동']
            
        for dong in dong_list:
            df = pickle.load(open('./data/{}/{} {}_{}_df.pkl'.format(gu,gu,dong,category),'rb'))
            train_df = pd.concat([train_df,df],axis=0)
    
    train_df = train_df.reset_index().drop(columns='index')
    train_df.index += 1
    return train_df

def make_test_dataset(gu_list, category):
    test_df = pd.DataFrame()
    
    for gu in gu_list:
        if gu=='광진구':
            dong_list=['화양동']
            
        for dong in dong_list:
            df = pickle.load(open('./data/{}/{} {}_{}_df.pkl'.format(gu,gu,dong,category),'rb'))
            test_df = pd.concat([test_df,df],axis=0)
    
    test_df = test_df.reset_index().drop(columns='index')
    test_df.index += 1
    return test_df

def seperate_positive_neutral_negative_point(df,column):
#     df[column] = df[column].astype('int')
    df.loc[df[column] == '2', column] = '1'    
    df.loc[df[column] == '4', column] = '5'
    return df

In [217]:
%%time

gu_list=['송파구','강남구','서초구']
category='치킨'

train_df = make_train_dataset(gu_list, category)
train_df = seperate_positive_neutral_negative_point(train_df,'Total')

CPU times: user 1.16 s, sys: 164 ms, total: 1.33 s
Wall time: 1.33 s


In [218]:
train_df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
186343,원엔원치킨-본점,bm**님,양념 치킨（1kg）/1,양은진짜 역대급으로 많네요 근데 먹고 아플꺼같은 느낌입니다 ...,1,2,5,3,2017년 12월 16일
186344,원엔원치킨-본점,wo**님,세트메뉴1（필로바베큐컴비네이션 R＋후라이드치킨）/1,"치킨도 괜찮고, 피자는 도우가 패스츄리라서 신기하네요. 맛있고 양도 괜찮았습니다.",5,5,5,4,2017년 12월 16일
186345,원엔원치킨-본점,dm**님,후라이드 치킨 （1kg）/2,치킨을 딱 본 순간 거뭇한 튀김옷에서 오래된 기름이 사용되었음을 알았습니다 치킨도 ...,5,5,5,5,2017년 12월 16일
186346,원엔원치킨-본점,pi**님,세트메뉴1（아라비안고로곤졸라 L＋후라이드치킨）/1,음... 여러가지 상호명을 쓰지만 한집인가보네요. 피자는 라지라고 적어놨지만 레귤러...,5,4,4,4,2017년 12월 14일
186347,원엔원치킨-본점,si**님,후라이드＋양념（2마리）/1,양념이랑.후라이드 맛있네요 배달도 빠르고 양이진짜 엄청많아요,5,5,5,5,2017년 12월 14일


In [219]:
train_df['Total'].value_counts()

5    158023
3     14459
1     13865
Name: Total, dtype: int64

## hannanum

## twitter

In [220]:
hannanum.nouns(train_df['Review'][1])

['양도작고', '고', '계란찜이너무작아서']

In [221]:
twitter.nouns(train_df['Review'][1])

['양도', '작고', '고기', '계란찜', '너무']

In [222]:
hannanum.morphs(train_df['Review'][1])

['양도작고',
 '고',
 '이',
 '기',
 '도',
 '딱딱하',
 '어',
 '고',
 '.....',
 '계란찜이너무작아서',
 '아쉽',
 '었어',
 '요',
 '....']

In [223]:
twitter.morphs(train_df['Review'][1])

['양도',
 '작고',
 '고기',
 '도',
 '딱딱하고',
 '.....',
 '계란찜',
 '이',
 '너무',
 '작아',
 '서',
 '아쉬웠',
 '어요',
 '....']

In [236]:
train_df

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
1,홍초불닭,fa**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(순한맛),추가 선택(치즈 추가),메뉴...",양도작고 고기도 딱딱하고..... 계란찜이너무작아서 아쉬웠어요....,1,2,1,4,4일 전
2,홍초불닭,ii**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(매운맛),메뉴 선택(양념구이불족발))",최악 새벽 3시 47분에 주문 오전 6시 4분에 배달 심지어 메뉴도 잘못... 2만...,1,1,1,1,2018년 8월 10일
3,홍초불닭,dd**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(중간맛),추가 선택(치즈 추가),메뉴...",오랜만에 맛 있네요 ^^,3,4,3,3,2018년 8월 5일
4,홍초불닭,손님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(중간맛),추가 선택(치즈 추가),메뉴...",음식도 많고 맛있어요,5,4,4,4,2018년 8월 2일
5,홍초불닭,js**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(중간맛),추가 선택(치즈 추가),메뉴...",쿨피스 2개 시켰는데 큰거 2개에 작은거 하나 더 주셨어욤ㅎㅎㅎ 무뼈닭발인데 뼈가 ...,5,4,4,4,2018년 7월 25일
6,홍초불닭,2g**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(순한맛),메뉴 선택(무뼈닭발))",너무나 맛있네여,5,4,4,4,2018년 7월 10일
7,홍초불닭,tb**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(순한맛),메뉴 선택(무뼈닭발))",닭발 밑에 파가 잇어 달달한맛이나서 확실히 호불호가 갈리는 맛이지만 제 입맛에는 괜...,5,4,4,4,2018년 6월 12일
8,홍초불닭,co**님,"세트（기본메뉴 小＋계란찜＋주먹밥）/1(맛 선택(순한맛),메뉴 선택(순살불닭))",양이나 배달은 좋았는데 맛이 제 취향이 아니네요...,3,3,4,4,2018년 5월 26일
9,홍초불닭,mi**님,"오돌뼈/1(사이즈 선택(小),맛 선택(매운맛)),주먹밥/2",야식으로 술 안주로 딱이네요ㅠㅎ,5,5,5,5,2018년 5월 17일
10,홍초불닭,yu**님,"무뼈닭발/1(사이즈 선택(小),맛 선택(순한맛),추가 선택(치즈 추가)),쿨피스/1",맛나요.......,3,4,4,4,2018년 5월 16일


---

# Test (방이2동 - 치킨)

In [224]:
%%time

gu_list=['광진구']
category='치킨'

test_df = make_test_dataset(gu_list, category)
test_df = seperate_positive_neutral_negative_point(test_df,'Total')

CPU times: user 86.5 ms, sys: 16.8 ms, total: 103 ms
Wall time: 102 ms


In [225]:
test_df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
21046,롯데리아-건대스타시티점,ch**님,"AZ（아재）버거 오리지널/1,양념감자/1(<시즈닝 선택>(시즈닝_오니언맛)),시즈닝...","성수점이 가깝지만, 요기요 서비스가 안되서 조금 멀어도 건대점을 이용했습니다. 거리...",3,,,,2016년 10월 11일
21047,롯데리아-건대스타시티점,ha**님,트윈B팩/1(<시즈닝 선택>(시즈닝_오니언맛)),"식지않고 따뜻하게 맛있게 왔어여!,",5,,,,2016년 10월 6일
21048,롯데리아-건대스타시티점,rl**님,"치킨버거 세트/1(<디저트 선택>(양념감자（오니언）),<드링크 선택>(세트 콜라（R...",좋아요~ 만족했습니다~,5,,,,2016년 9월 23일
21049,호식이두마리치킨-화양점,kj**님,후라이드치킨＋양념소스치킨＋콜라/1(치킨 선택(순살＋순살（1kg）)),"먹기 좋은 크기에 양,배달,맛 완전 훌륭해요~~",5,5,5,5,14시간 전
21050,호식이두마리치킨-화양점,ai**님,양념소스치킨＋콜라/1(치킨선택 1(뼈있는치킨)),맛나게 잘먹었어요.,5,4,4,5,2일 전


In [226]:
test_df['Total'].value_counts()

5    17804
1     1661
3     1585
Name: Total, dtype: int64

---

## Naive Bayes Model

In [243]:
X = train_df['Review']
len(X)

186347

In [244]:
y = train_df['Total']
len(y)

186347

In [245]:
MNB_model = Pipeline([
            ('vect', CountVectorizer()), 
            ('mb', MultinomialNB()),
        ])

In [246]:
%time MNB_model.fit(X, y)

CPU times: user 2.41 s, sys: 74.6 ms, total: 2.48 s
Wall time: 2.49 s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('mb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [239]:
X_test = test_df['Review']
len(X_test)

21050

In [240]:
y_test = test_df['Total']
len(y_test)

21050

In [233]:
print(accuracy_score(y_test, MNB_model.predict(X_test)))

0.8769121140142517


In [234]:
print(confusion_matrix(y_test, MNB_model.predict(X_test)))

[[  807   123   731]
 [  153   176  1256]
 [  158   170 17476]]


In [235]:
print(classification_report(y_test, MNB_model.predict(X_test)))

             precision    recall  f1-score   support

          1       0.72      0.49      0.58      1661
          3       0.38      0.11      0.17      1585
          5       0.90      0.98      0.94     17804

avg / total       0.84      0.88      0.85     21050



In [29]:
result = pd.DataFrame(MNB_model.predict(X_test), y_test).reset_index()
result.columns = ['Pred','Real']
# result[result['Pred']!=result['Real']]

---

## XGB

In [143]:
XGB_model = Pipeline([
            ('vect', CountVectorizer()), 
            ('xgb', xgboost.XGBClassifier(max_depth=30, n_estimators=300, n_jobs=6)),
        ])

In [144]:
%time XGB_model.fit(X, y)

CPU times: user 16min 59s, sys: 1.71 s, total: 17min 1s
Wall time: 17min 2s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [145]:
print(accuracy_score(y_test, XGB_model.predict(X_test)))

0.868978622327791


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [146]:
print(confusion_matrix(y_test, XGB_model.predict(X_test)))

[[  584   102   975]
 [  114   141  1330]
 [  110   127 17567]]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [147]:
print(classification_report(y_test, XGB_model.predict(X_test)))

             precision    recall  f1-score   support

          1       0.72      0.35      0.47      1661
          3       0.38      0.09      0.14      1585
          5       0.88      0.99      0.93     17804

avg / total       0.83      0.87      0.84     21050



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [222]:
print(accuracy_score(y_test, XGB_model.predict(X_test)))

0.871858446973966


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [223]:
print(confusion_matrix(y_test, XGB_model.predict(X_test)))

[[ 195   48  332]
 [  48   72  542]
 [  70   97 7469]]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [224]:
print(classification_report(y_test, XGB_model.predict(X_test)))

             precision    recall  f1-score   support

          1       0.62      0.34      0.44       575
          3       0.33      0.11      0.16       662
          5       0.90      0.98      0.93      7636

avg / total       0.84      0.87      0.85      8873



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


---